In [1]:
%run 1_import_analysis.ipynb
%run 2_preprocessing.ipynb

'Cargando el dataset...'

'El dataset tiene 1599999 filas y 6 columnas'

,target,id,date,flag,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew



--------------------------------------------------

Corroborando valores nulos y duplicados
Nulls: 
target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

--------------------------------------------------

Duplicates:  0

--------------------------------------------------

Eliminando columnas innecesarias


'Distribución de la variable target:'

positive    800000
negative    799999
Name: target, dtype: int64


--------------------------------------------------

Train-test split:
X_train:  (1119999,)
X_test:  (480000,)
y_train:  (1119999,)
y_test:  (480000,)

--------------------------------------------------



## Configuración del pipeline y modelos a utilizar para la predicción de los sentimientos

In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [3]:
from sklearn.feature_selection import SelectKBest, chi2

In [4]:
common_steps = [
            ('text_processing', TextProcessing()),
           ('vectorizer', CountVectorizer()),
            ('tfidf', TfidfTransformer()),
          # ('feature_selection', SelectKBest(chi2, k=1000)),
            ]


In [5]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV,StratifiedKFold

In [6]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from xgboost.sklearn import XGBClassifier


In [7]:
# Modelos a utilizar

LR_pipe = [Pipeline( common_steps + [('lr', LogisticRegression())]), {'lr__C':[0.01,0.1,1], 'lr__penalty':['l2']}]
DT_pipe = [Pipeline( common_steps + [('dt', tree.DecisionTreeClassifier())]), {'dt__max_depth':[  512,1024 ] }]
MNB_pipe = [Pipeline( common_steps + [('mnb', MultinomialNB())]), {'mnb__alpha':[1, 10, 20]}]
RF_pipe = [Pipeline( common_steps + [('rf', RandomForestClassifier())]), {'rf__n_estimators':[  100, 200], 'rf__max_depth':[   5,10] }]
XGB_pipe = [Pipeline( common_steps + [('xgb', XGBClassifier())]), {'xgb__n_estimators':[  100, 200], 'xgb__max_depth':[   5,10] }]

skf=StratifiedKFold(n_splits=3,random_state=0,shuffle=True)
models = []


In [8]:
pipelines = [LR_pipe, DT_pipe, MNB_pipe, RF_pipe]

for pipe in pipelines:
    print('fitting',pipe[0].steps[-1][0])
    GS_CV=GridSearchCV(pipe[0],pipe[1],cv=skf,verbose=10,n_jobs=3);
    GS_CV.fit(X_train, y_train);
    models.append(GS_CV)
    print('best score:',GS_CV.best_score_)
    print('best params:',GS_CV.best_params_) 
    separator()

fitting lr
Fitting 3 folds for each of 3 candidates, totalling 9 fits


c:\Users\LuisD\anaconda3\envs\dh\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


best score: 0.777047122363502
best params: {'lr__C': 1, 'lr__penalty': 'l2'}

--------------------------------------------------

fitting dt
Fitting 3 folds for each of 2 candidates, totalling 6 fits


In [ ]:
## Models evaluation

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


for model in models:
    print(model.best_estimator_)
    y_pred = model.predict(X_test)
    print('Accuracy: ', accuracy_score(y_test, y_pred))
    print('Classification report: ', classification_report(y_test, y_pred))
    print('Confusion matrix: ', confusion_matrix(y_test, y_pred))



In [ ]:
# pick best model
best_model = models[0]
for model in models:
    if model.best_score_ > best_model.best_score_:
        best_model = model


best_model        
        